# Sparklehorse - Der SQL-Chatbot der Magpie

## 1. Vorbereitung

In einem ersten Schritt definieren wir unser Arbeitsverzeichnis. 

In [2]:
import os
os.getcwd()
os.chdir("c:/Users/mhu/Documents/gitHub/magpie_chatbot")

Wir laden unsere Umgebungsvariablen (inkl. OpenAI-API-Key) und initialisiere den Chatbot mit dem Modell "gpt-4o" von OpenAI.

In [3]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
llm = ChatOpenAI(model="gpt-4o")

  Im folgenden stellen wir Verbindung zur Magpie her.

In [4]:
import pandas as pd
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit 

db = SQLDatabase.from_uri("duckdb:///data/view_magpie.db")

c:\Users\mhu\AppData\Local\miniconda3\envs\chatbot_magpie\lib\site-packages\duckdb_engine\__init__.py:184: DuckDBEngineWarning: duckdb-engine doesn't yet support reflection on indices
  warnings.warn(


## 2. Tools

### 2.1 Standardisierte Langchain Tools

Wir initialisieren ein standardisiertes Toolkit. Es stellt Funktionen bereit, um SQL-Queries über natürliche Sprache zu erzeugen und auszuführen. Wir lassen uns Namen und Funktion der standardisierten Tools anzeigen:

In [5]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

for tool in tools:
    print(f"Tool Name: {tool.name}")
    print(f"Description: {tool.description}")
    print("-" * 40)

Tool Name: sql_db_query
Description: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.
----------------------------------------
Tool Name: sql_db_schema
Description: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3
----------------------------------------
Tool Name: sql_db_list_tables
Description: Input is an empty string, output is a comma-separated list of tables in the database.
----------------------------------------
Tool Name: sql_db_query_checker
Description: Use this tool to double check if your query is

Die Standardtools aus`SQLDatabaseToolkit` können also die folgenden Standardfunktionen übernehmen: 

- `sql_db_query`  
  Führt eine übergebene SQL-Abfrage aus. Gibt das Ergebnis oder eine Fehlermeldung zurück. Bei Fehlern wie „Unknown column“ sollte zuvor das Tabellenschema geprüft werden.

- `sql_db_schema`  
  Gibt das Schema (Spaltennamen und -typen) sowie Beispielzeilen für angegebene Tabellen zurück. Vorher sollte geprüft werden, ob die Tabellen existieren.

- `sql_db_list_tables`  
  Listet alle Tabellen in der verbundenen Datenbank auf.

- `sql_db_query_checker`  
  Prüft eine SQL-Abfrage auf syntaktische Korrektheit, bevor sie mit sql_db_query ausgeführt wird. Sollte immer vorher verwendet werden.


### 2.2 Maßgeschneiderte Langchain Tools

#### Retriever `rt_beschr_variable`

`rt_beschr_variable` erlaubt die semantischen Suche über Werte aus einer Datenbankspalte. 

1. Wir sammeln sämtliche Unique Werte aus `beschr_variable` und wandeln diese mit OpenAIs Embeddings-Methdode `text-embedding-3-large` in Embeddings um. Die werden in einen Vektorstore gesichert.
2. Der Vektorstore wird in einen Retriever umgewandelt, der bei einer Anfrage die `n=10` ähnlichsten Begriffe zurückgibt.
3. Schließlich wird mit `create_retriever_tool` ein Tool erzeugt, das den Retriever kapselt. Dieses Tool kann von Sparklehorse genutzt werden, um Benutzereingaben mit unsicherer Schreibweise oder unvollständigen Begriffen mit den tatsächlichen Werten in der Datenbank abzugleichen.

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.agents.agent_toolkits import create_retriever_tool
import ast
import re

def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [string.strip() for string in res]
    return list(set(res))


beschr_variable = query_as_list(db, "SELECT variable_beschr FROM view_daten_reichweite_menge")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_texts(beschr_variable)

retriever_beschr_variable  = vector_store.as_retriever(search_kwargs={"k": 10})

description = (
    "Verwenden, um Werte für Filterabfragen nachzuschlagen. Die Eingabe ist eine ungefähre Schreibweise "
    "eines Eigennamens, die Ausgabe sind gültige Eigennamen. Verwende den Begriff, der der Eingabe am ähnlichsten ist."
)

rt_beschr_variable = create_retriever_tool(
    retriever_beschr_variable,
    name="rt_beschr_variable",
    description=description,
)

Wir testen nun den Retriever:

In [7]:
def print_clean_result(result):
    print("\n".join(result.split("\n\n")))

result = rt_beschr_variable.invoke("Studienanfänger ohne Abitur")

print_clean_result(result)

Studienabsolventen ohne Abitur
Anzahl der Studienanfänger ohne Abitur
Anteil der Studienanfänger ohne Abitur
Anteil der Studienabsolventen ohne Abitur
Studienanfänger 1. FS Lehramt nach angestr. Prüfungen Bachelor + Staatsexamen im 1. Hochschulsemester
Studienabsolventen Bildungsausländer
Studienabsolventen im Weiterbildungsstudium
Studienanfänger 1. FS Lehramt nach angestr. Prüfungen Bachelor + Staatsexamen (ges. Studienjahr)
Studierende
Studienabsolventen Bildungsinländer


#### Retriever `rt_reichweite_variable`

Wir bauen einen ähnlichen Retriever nun auch für die Variablen `reichweite_beschr_list`. Allerdings begrenzen wir die uniquen Reichweiten auf jene einer Variable.

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def invoke_agent_with_check(question: str):
    result = agent_executor.invoke({"messages": [HumanMessage(content=question)]})
    messages = result["messages"]
    last_message = messages[-1]

    if isinstance(last_message, AIMessage) and "[USER_CLARIFICATION_NEEDED]" in last_message.content:
        rückfrage = last_message.content.replace("[USER_CLARIFICATION_NEEDED]", "").strip()
        return f"⚠️ Rückfrage: {rückfrage}"
    else:
        return last_message.content

# Few Shot


In [ ]:
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

reichweiten_beispiele = [
    {"frage": "Wie viele Absolventen für Berufliche Schulen gab es?", "variable_beschr": "Anzahl der Absolventen für Berufliche Schulen", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war die Studierquote bildungsferner Schichten?", "variable_beschr": "Studierquote bildungsferne Schichten", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele dauerhaft eingestellte Lehrkräfte (inkl. Seiteneinsteigern, ohne Referendare) gab es?", "variable_beschr": "Anzahl dauerhaft eingestellte Lehrkräfte (inkl. Seiteneinsteigern, ohne Referendare)", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war der Handlungsfeldindex: Lehrer Bildung?", "variable_beschr": "Handlungsfeldindex: Lehrer Bildung", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele Universitätsschulverbünde gab es?", "variable_beschr": "Anzahl Universitätsschulverbünde", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war der Anteil berufsbegleitender Master?", "variable_beschr": "Anteil berufsbegleitender Master", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele Studienabsolventen T gab es?", "variable_beschr": "Studienabsolventen T", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch waren die internen FuE-Aufwendungen?", "variable_beschr": "Interne FuE-Aufwendungen", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch war der Anteil der männlichen Grundschullehramtsstudierenden?", "variable_beschr": "Anteil der männlichen Grundschullehramtsstudierende", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie viele Studienabsolventen im Weiterbildungsstudium gab es?", "variable_beschr": "Studienabsolventen im Weiterbildungsstudium", "reichweite_beschr_list": "Deutschland"},
    {"frage": "Wie hoch waren die Drittmittel vom Bund 2021 in Deutschland?", "variable_beschr": "Drittmittel vom Bund", "reichweite_beschr_list": "Deutschland"}
]

example_prompt = PromptTemplate(
    input_variables=["frage", "variable_beschr", "reichweite_beschr_list"],
    template="Frage: {frage}\nVariable: {variable_beschr}\n→ Reichweite: {reichweite_beschr_list}"
)

reichweite_prompt = FewShotPromptTemplate(
    examples=reichweiten_beispiele,
    example_prompt=example_prompt,
    prefix="Wähle aus den möglichen Reichweiten die beste. Nutze 'Deutschland', wenn keine Region, Organisation o. Ä. genannt wird.",
    suffix="Frage: {frage}\nVariable: {variable_beschr}\nKandidaten:\n{kandidaten}\n→ Reichweite:",
    input_variables=["frage", "variable_beschr", "kandidaten"]
)

In [80]:
from langchain_core.tools import tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

@tool
def variable_beschr(user_question: str) -> str:
    """
    Nutzt ein LLM und Embeddings, um aus der Frage eine passende Variable zu bestimmen
    und gibt dann die exakte Variable aus der Datenbank zurück.
    """
    docs = retriever_beschr_variable.get_relevant_documents(user_question)
    if not docs:
        return "Error: Keine passende Variable gefunden."

    kandidaten = "\n".join(f"- {doc.page_content.strip()}" for doc in docs)
    print(kandidaten)

    auswahl_prompt = PromptTemplate(
        input_variables=["frage", "kandidaten"],
        template="""
    Wähle exakt **eine** der folgenden Variablen, die am besten zur Frage passt.
    Wähle **nur dann** eine Variable aus, wenn sie **exakt** zur Frage passt.
    Nutze **keine verwandten Begriffe**, Oberkategorien oder Synonyme.
    Gib den Text **genau so** zurück, wie er bei den Kandidaten steht.

    Frage: {frage}

    Kandidaten:
    {kandidaten}

    Beste Variable:
    """
    )
    auswahl_chain = auswahl_prompt | llm
    best_match = auswahl_chain.invoke({
        "frage": user_question,
        "kandidaten": kandidaten
    }).content.strip()

    query = f"""
        SELECT variable_beschr 
        FROM view_daten_reichweite_menge 
        WHERE variable_beschr = '{best_match}' 
        LIMIT 1;
    """
    result = db.run_no_throw(query)
    if result:
        return result
    else:
        return "[USER_CLARIFICATION_NEEDED] Ich konnte keine passende Variable finden. Bitte geben Sie die gewünschte Variable genauer an."



import re
@tool
def get_reichweite_beschr_list(user_question: str) -> str:
    """
    Ermittelt eine passende Reichweite (z. B. Region, Organisation, etc.), basierend auf der
    zur Frage gehörigen Variable und den verfügbaren Einträgen in der Datenbank.
    """
    print("[DEBUG] Eingabe-Frage:", user_question)

    raw_variable = variable_beschr.run(user_question)
    print("[DEBUG] raw_variable:", raw_variable)

    match = re.search(r"'([^']+)'", str(raw_variable))
    if not match:
        print("[DEBUG] Abbruch: Keine gültige Variable extrahiert")
        return "Fehler: Konnte keine gültige Variable bestimmen."

    variable = match.group(1)
    print("[DEBUG] bereinigte variable:", variable)

    if "Error" in variable:
        return "Fehler: Konnte keine gültige Variable bestimmen."

    escaped_variable = variable.replace("'", "''")
    print("[DEBUG] escaped_variable:", escaped_variable)

    query = f"""
        SELECT DISTINCT reichweite_beschr_list 
        FROM view_daten_reichweite_menge 
        WHERE variable_beschr = '{escaped_variable}'
    """
    print("[DEBUG] SQL-Abfrage gültige_reichweiten:", query)
    gültige_reichweiten = query_as_list(db, query)
    print("[DEBUG] gültige_reichweiten:", gültige_reichweiten)

    if not gültige_reichweiten:
        return "[USER_CLARIFICATION_NEEDED] Ich konnte keine passende Reichweite ermitteln. Bitte präzisieren Sie, welche Region oder Organisation gemeint ist."


    vector_store = InMemoryVectorStore(OpenAIEmbeddings(model="text-embedding-3-large"))
    _ = vector_store.add_texts(gültige_reichweiten)
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})

    top_matches = retriever.get_relevant_documents(user_question)
    reichweiten_kandidaten = [doc.page_content for doc in top_matches]
    print("[DEBUG] Top 5 Reichweiten-Kandidaten:", reichweiten_kandidaten)

    kandidaten_text = "\n".join(reichweiten_kandidaten)

    llm_chain = reichweite_prompt | llm
    best_match = llm_chain.invoke({
        "frage": user_question,
        "variable_beschr": variable,
        "kandidaten": kandidaten_text
    }).content.strip()

    print("[DEBUG] LLM-best_match:", best_match)

    # Validierung: nur erlaubte Rückgabe
    if best_match not in gültige_reichweiten:
        print(f"[DEBUG] LLM-Match ungültig ('{best_match}'), Rückfrage erforderlich")
        return "[USER_CLARIFICATION_NEEDED] Ich konnte keine passende Reichweite ermitteln. Bitte konkretisieren Sie Ihre Anfrage."
        
    query = f"""
        SELECT reichweite_beschr_list 
        FROM view_daten_reichweite_menge 
        WHERE reichweite_beschr_list = '{best_match}' 
        LIMIT 1;
    """
    print("[DEBUG] SQL-Abfrage finale Auswahl:", query)
    result = db.run_no_throw(query)
    print("[DEBUG] Ergebnis:", result)

    return result if result else "Error: Keine passende Reichweite gefunden."



tools.extend([variable_beschr, get_reichweite_beschr_list])

In [ ]:
test_input = "Wie hoch waren die externe fue-aufwendungen, im Jahr 2020, in Deutschland, im Sektor wirtschaftssektor, bei forschungsintensive wirtschaftszweige Forschung?"
output = variable_beschr(test_input)
print(output)

In [ ]:
test_input = "Wie hoch waren die drittmittel vom bund, im Jahr 2021, in Deutschland?"
output = get_reichweite_beschr_list(test_input)
print(output)

In [ ]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1, "Die Anzahl der Nachrichten im Template ist nicht 1!"
# Bearbeite die bestehende Nachricht, indem du Text hinzufügst
prompt_template.messages[0].prompt.template += (
    "\nYou are Sparklehorse, a chatbot for the Stifterverband organization. "
    "Your primary task is to answer questions related to the Magpie database."
)

prompt_template.messages[0].pretty_print()

In [ ]:
system_message = prompt_template.format(
    dialect=db.dialect, 
    top_k=5
)

print(system_message)

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

# Systemnachricht mit extra Anweisungen
suffix = (
    "Bevor du eine SQL-Abfrage generierst, beachte bitte folgende Regeln strikt:\n"
    "1. Nutze das Tool `variable_beschr`, um die korrekte Variable aus der Nutzerfrage zu bestimmen. Verwende ausschließlich den exakten Rückgabewert dieses Tools für `variable_beschr` in der SQL-Abfrage.\n"
    "2. Nutze das Tool `get_reichweite_beschr_list`, um die passende Reichweite zu ermitteln. Verwende ausschließlich den Rückgabewert dieses Tools für `reichweite_beschr_list` in der SQL-Abfrage.\n"
    "3. Verwende **niemals** andere Felder wie `tag_list` oder `LIKE`-Abfragen. Nutze **immer exakte Vergleiche** mit `=`.\n"
    "4. Verwende ausschließlich die Tabelle `view_daten_reichweite_menge` für alle Abfragen.\n"
    "5. Falls ein Jahr in der Frage genannt wird, filtere mit `date_part('year', zeit_start) = <Jahr>`.\n"
    "6. Berücksichtige die Spalte `wert_einheit`, z. B. 'in Tsd. Euro', 'Anzahl', 'Prozent', 'VZÄ', 'Mitarbeiter'.\n"
    "7. Gib immer die finale SQL-Abfrage vollständig aus und erkläre sie. Rate niemals IDs oder Werte.\n"
    "8. Falls keine passende Variable oder Reichweite gefunden wurde, rate nicht irgendwelche Werte. \n"
    "9. Stelle sicher, dass Antworttext und SQL-Abfrage immer auf den gleichen `variable_beschr`- und `reichweite_beschr_list`-Werten basieren, um Konsistenz zu gewährleisten.\n"
    "10. Verwende in deiner Antwort exakt die Begriffe, die du in der SQL-Abfrage benutzt hast. Nutze insbesondere den Wert aus `reichweite_beschr_list` vollständig im Antwortsatz. Beispiel: Wenn `reichweite_beschr_list = 'Wirtschaftssektor | Deutschland'`, schreibe: 'im Wirtschaftssektor in Deutschland'.\n"
    "11. Beantworte ausschließlich Fragen zur Magpie-Datenbank. Wenn eine Frage nichts mit den Variablen, Reichweiten oder Daten aus der Magpie-Datenbank zu tun hat, antworte mit: 'Ich kann nur Fragen zur Magpie-Datenbank beantworten.'"

)

 
system = f"{system_message}\n\n{suffix}"

# Neuen ReAct-Agent erstellen mit den vollständigen Tools
agent_executor = create_react_agent(llm, tools, state_modifier=system)


In [ ]:
def stream_agent_with_check(question: str):
    stream = agent_executor.stream({"messages": [HumanMessage(content=question)]}, stream_mode="values")
    for step in stream:
        msg = step["messages"][-1]
        if "[USER_CLARIFICATION_NEEDED]" in msg.content:
            rückfrage = msg.content.replace("[USER_CLARIFICATION_NEEDED]", "").strip()
            print(f"⚠️ Rückfrage: {rückfrage}")
            break
        else:
            msg.pretty_print()

In [ ]:
# Testanfrage an den Agenten
question = "Wie hoch waren die externe fue-aufwendungen, im Jahr 2020, in Deutschland, im Sektor wirtschaftssektor, bei forschungsintensive wirtschaftszweige Forschung?"

stream_agent_with_check(question)

In [ ]:
import pandas as pd
from langchain.schema import HumanMessage

df = pd.read_excel("data/test_quest_mixed_nr_5.xlsx").sample(n=20, random_state=1)
 
for question in df["Frage"].dropna():
    result = agent_executor.invoke({"messages": [HumanMessage(content=question)]})
    messages = result["messages"]
    antwort = messages[-1].content
    print(f"\nFrage: {question}\nAntwort: {antwort}")


In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []

def ask_with_memory(user_input):
    chat_history.append(HumanMessage(content=user_input))
    response = agent_executor.invoke({"messages": chat_history})
    reply = response["messages"][-1]
    chat_history.append(reply)
    return reply.content

In [ ]:
print(ask_with_memory("Wie hoch waren die FuE-Ausgaben 2022 in Deutschland?"))
print(ask_with_memory("Und im Jahr davor?"))

In [ ]:
print(ask_with_memory("Wer ist Norbert Elias?"))